# Vicuna 4 bit quantized

In [1]:
import sys
import os
sys.path.append("../src")
import llm_utils
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

classes = ["War/Terror", "Conspiracy Theory", "Education", "Election Campaign", "Environment", 
              "Government/Public", "Health", "Immigration/Integration", 
              "Justice/Crime", "Labor/Employment", 
              "Macroeconomics/Economic Regulation", "Media/Journalism", "Religion", "Science/Technology"]

# Without context and classification only
## Example:
    ### Human: Classify the Tweet based on if it's about {label}. Use 1 or 0 as class.\n\nTweet: {tweet_text}\n### Assistant:\nClass = 
without_context_classification_only_df = pd.read_csv("../data/vicuna_4bit/generic_prompt_without_context_only_classification/generic_test_0.csv")
extraction_function = llm_utils.get_extraction_function("extract_first_character")
prediction_per_class, confusion_matrices, classification_reports = llm_utils.calculate_binary_metrics(without_context_classification_only_df, classes, extraction_function)
without_context_classification_only = {"confusion_matrices": confusion_matrices, "classification_reports": classification_reports}

# without context and elaboration first
## Example:
    ### Human: Elaborate on whether you think the Tweet is about {label} or something else.\nTweet: {tweet_text}\n### Assistant:\nElaboration: 
    #Followup :\n Assign class 1 for {label} or 0 for not. \n###Assistant:\nClass:  
without_context_elaboration_first_df = pd.read_csv("../data/vicuna_4bit/generic_prompt_without_context_elaboration_first/generic_test_0.csv")
extraction_function = llm_utils.get_extraction_function("extract_last_float")
prediction_per_class, confusion_matrices, classification_reports = llm_utils.calculate_binary_metrics(without_context_elaboration_first_df, classes, extraction_function)
without_context_elaboration_first = {"confusion_matrices": confusion_matrices, "classification_reports": classification_reports}

#with rules as context and classification only
## Example:
    ### Human: Based on rules, classify the Tweet based on if it's about {label}. Use 1 or 0 as class.\n\nRules: {rules}\nTweet: {tweet_text}\n### Assistant:\nClass:
with_rules_classification_only_df = pd.read_csv("../data/vicuna_4bit/generic_prompt_with_rules_only_classification/generic_test_0.csv")
extraction_function = llm_utils.get_extraction_function("extract_second_character")
prediction_per_class, confusion_matrices, classification_reports = llm_utils.calculate_binary_metrics(with_rules_classification_only_df, classes, extraction_function)
with_rules_classification_only = {"confusion_matrices": confusion_matrices, "classification_reports": classification_reports}

# with rules as context and elaboration first
## Example:
    ### Human: Based on rules, elaborate whether you think the Tweet is about {label}.\nRules: {rules}\nTweet: {tweet_text}\n### Assistant:\nElaboration: 
    #Followup :\n Assign class 1 for {label} or 0 for not. \n###Assistant:\nClass: 
with_rules_elaboration_first_df = pd.read_csv("../data/vicuna_4bit/generic_prompt_with_rules_elaboration_first/generic_test_0.csv")
extraction_function = llm_utils.get_extraction_function("extract_first_character")
prediction_per_class, confusion_matrices, classification_reports = llm_utils.calculate_binary_metrics(with_rules_elaboration_first_df, classes, extraction_function)
with_rules_elaboration_first = {"confusion_matrices": confusion_matrices, "classification_reports": classification_reports}

TypeError: 'NoneType' object is not callable

In [22]:
import sys
import os
sys.path.append("../src")
import llm_utils
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import ast
import numpy as np


def classification_reports_to_df(classification_reports, binary):

    if binary:
        # Your code for creating the DataFrame and adding the results
        df = pd.DataFrame(columns=['label', 'f1_score_macro', #'precision_macro', 'recall_macro', 'support_macro',
                                        'f1_score_class_0','support_class_0',
                                        'f1_score_class_1', 'support_class_1'])

        for label, cr in classification_reports.items():
            try: 
                df = df.append({
                    'label': label,
                    'f1_score_macro': cr['macro avg']['f1-score'],
                    #'precision_macro': cr['macro avg']['precision'],
                    #'recall_macro': cr['macro avg']['recall'],
                    #'support_macro': cr['macro avg']['support'],
                    'f1_score_class_0': cr['0']['f1-score'],
                    #'precision_class_0': cr['0']['precision'],
                    #'recall_class_0': cr['0']['recall'],
                    'support_class_0': cr['0']['support'],
                    'f1_score_class_1': cr['1']['f1-score'],
                    #'precision_class_1': cr['1']['precision'],
                    #'recall_class_1': cr['1']['recall'],
                    'support_class_1': cr['1']['support']
                }, ignore_index=True)
            except Exception as e:
                print(f"Error for {label}: {e}")
                df = df.append({
                    'label': label,
                    'f1_score_macro': None,
                    #'precision_macro': None,
                    #'recall_macro': None,
                    #'support_macro': None,
                    'f1_score_class_0': None,
                    #'precision_class_0': None,
                    #'recall_class_0': None,
                    'support_class_0': None,
                    'f1_score_class_1': None,
                    #'precision_class_1': None,
                    #'recall_class_1': None,
                    'support_class_1': None
                }, ignore_index=True)
                continue

        # Display the results
        return df
    
    return None

def extract_multilabel_list(classification_str, classes):
    if classification_str is np.nan:
        return ["Others"]
    classification_str = classification_str.replace("[", "").replace("]", "").replace("'", "")
    classification_str = classification_str.split(", ")
    for class_ in classification_str:
        if class_ not in " ".join(classes):
            #print(class_)
            classification_str.remove(class_)
    if classification_str == []:
        classification_str = ["Others"]
    return list(set(classification_str))

def extract_multilabel_list_only_first_class(classification_str, classes):
    if classification_str is np.nan:
        return ["Others"]
    classification_str = classification_str.replace("[", "").replace("]", "").replace("'", "")
    classification_str = classification_str.split(", ")
    for class_ in classification_str:
        if class_ not in " ".join(classes):
            #print(class_)
            classification_str.remove(class_)
    if classification_str == []:
        classification_str = ["Others"]
    classification_str = [classification_str[0]]
    return list(set(classification_str))

def to_dataframe(average_reports, classes):
    data = {
        "precision": [],
        "recall": [],
        "f1-score": [],
        "support": []
    }
    index = []

    for i, average_report in enumerate(average_reports):
        for class_name, metrics in average_report.items():
            if class_name in {'micro avg', 'macro avg', 'weighted avg', 'accuracy'}:
                continue
            index.append(classes[i])
            data["precision"].append(metrics["0"]["precision"])
            data["recall"].append(metrics["0"]["recall"])
            data["f1-score"].append(metrics["0"]["f1-score"])
            data["support"].append(metrics["0"]["support"])

    return pd.DataFrame(data, index=index)

classes = ["War/Terror", "Conspiracy Theory", "Education", "Election Campaign", "Environment", 
              "Government/Public", "Health", "Immigration/Integration", 
              "Justice/Crime", "Labor/Employment", 
              "Macroeconomics/Economic Regulation", "Media/Journalism", "Religion", "Science/Technology", "Others"]
    
#df1 = pd.read_csv("../data/vicuna_4bit/lora/multilabel_with_rules/valid_generic_test_0.csv")
df2 = pd.read_csv("../data/vicuna_4bit/lora/multilabel_without_context_v01/test_generic_test_0.csv")
vicuna_lora_multilabel_without_context_v01_df = df2#pd.concat([df1, df2])
# Cleaning and extracting multilabel classes
vicuna_lora_multilabel_without_context_v01_df['annotations'] = vicuna_lora_multilabel_without_context_v01_df['annotations'].apply(lambda x: extract_multilabel_list(x, classes))
vicuna_lora_multilabel_without_context_v01_df['testmultilabel_without_context_v01'] = vicuna_lora_multilabel_without_context_v01_df['testmultilabel_without_context_v01'].apply(lambda x: extract_multilabel_list(x, classes))

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

# Transform the labels into binary format
mlb = MultiLabelBinarizer(classes=classes)

# Prepare ground truth and predictions
y_true = mlb.fit_transform(vicuna_lora_multilabel_without_context_v01_df['annotations'])
y_pred = mlb.transform(vicuna_lora_multilabel_without_context_v01_df['testmultilabel_without_context_v01'])

#import pandas as pd
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(y_true, y_pred, output_dict=True, target_names=classes)

# Convert the report to a pandas DataFrame
df_report = pd.DataFrame(report).transpose()

print(df_report)

                                    precision    recall  f1-score  support
War/Terror                           0.500990  0.992157  0.665789    255.0
Conspiracy Theory                    0.368421  0.777778  0.500000     45.0
Education                            0.322581  0.769231  0.454545     13.0
Election Campaign                    0.766667  0.696970  0.730159     33.0
Environment                          0.458333  0.785714  0.578947     14.0
Government/Public                    0.572082  0.859107  0.686813    291.0
Health                               0.345455  0.826087  0.487179     46.0
Immigration/Integration              0.315217  0.805556  0.453125     36.0
Justice/Crime                        0.413681  0.927007  0.572072    137.0
Labor/Employment                     0.243590  0.678571  0.358491     28.0
Macroeconomics/Economic Regulation   0.212996  0.951613  0.348083     62.0
Media/Journalism                     0.102564  0.916667  0.184486     48.0
Religion                 

/media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/Documents/ip6-twitter-disinformation/.conda/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['China', 'Civilian Casualties', 'Diplomacy', 'Entertainment', 'Event', 'Expression', 'Festival', 'Foreign Relations', 'Generosity', 'Genocide', 'Israel/Palestine', 'Judaism', 'Kurdish Forces', 'Military', 'Miscellaneous', 'Music', 'Nightlife', 'Optical Disk', 'Pandemics', 'Party', 'Party and Celebration', 'Partying', 'Performance', 'Political Platform', 'Politics', 'President', 'Productivity', 'Propaganda', 'Referendum...', 'Syria', 'Television', 'U.S.', 'US Domestic Policy', 'US International Relations', 'USA', 'United States', 'Voting', 'Wages/Labors', 'War', 'Welfare/Development', 'Welfare/Social Macroregulation', 'Wildlife', 'World Order', 'Youth'] will be ignored
  warnings.warn(


In [23]:
df2 = pd.read_csv("../data/vicuna_4bit/lora/multilabel_with_rules_v02/test_generic_test_0.csv")
vicuna_lora_multilabel_without_context_v01_df = df2#pd.concat([df1, df2])
# Cleaning and extracting multilabel classes
vicuna_lora_multilabel_without_context_v01_df['annotations'] = vicuna_lora_multilabel_without_context_v01_df['annotations'].apply(lambda x: extract_multilabel_list(x, classes))
vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02'] = vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02'].apply(lambda x: extract_multilabel_list(x, classes))

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

# Transform the labels into binary format
mlb = MultiLabelBinarizer(classes=classes)

# Prepare ground truth and predictions
y_true = mlb.fit_transform(vicuna_lora_multilabel_without_context_v01_df['annotations'])
y_pred = mlb.transform(vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02'])

#import pandas as pd
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(y_true, y_pred, output_dict=True, target_names=classes)

# Convert the report to a pandas DataFrame
df_report = pd.DataFrame(report).transpose()

print(df_report)

                                    precision    recall  f1-score  support
War/Terror                           0.386018  0.996078  0.556407    255.0
Conspiracy Theory                    0.340206  0.733333  0.464789     45.0
Education                            0.191489  0.692308  0.300000     13.0
Election Campaign                    0.628571  0.666667  0.647059     33.0
Environment                          0.190476  0.857143  0.311688     14.0
Government/Public                    0.547521  0.910653  0.683871    291.0
Health                               0.271605  0.956522  0.423077     46.0
Immigration/Integration              0.222973  0.916667  0.358696     36.0
Justice/Crime                        0.354223  0.948905  0.515873    137.0
Labor/Employment                     0.132075  0.750000  0.224599     28.0
Macroeconomics/Economic Regulation   0.180380  0.919355  0.301587     62.0
Media/Journalism                     0.095918  0.979167  0.174721     48.0
Religion                 

/media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/Documents/ip6-twitter-disinformation/.conda/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['', '#Collision kills taxi driver near Petroport on Queen Nandi Dr', '#NoCandidates = ‘ Government/Public ', '#Olympics', '#TeamUWI #TrendingNow', '@USER', 'Accident', 'Analysis', 'Architecture', 'Asia', 'Aviation', 'Behna', 'Brexit', 'Civilian', 'Collaboration', 'Conspiracy', 'Cuba', 'Death', 'Defeating_ISIS', 'Energy', 'Fake News', 'Geopolitics', 'Govern', 'Government/', 'Haftar', 'Happy birthday!', 'International Relations', 'Jabhat_al_Nusra', 'Justice', 'Justice/Cr', 'Labor/Emp', 'Leader', 'Mac', 'Macroeconom', 'Macroeconomics', 'Macroeconomics/', 'Mammals', 'Media_coverage_of_war', 'Military', 'Military_group', 'Miscellaneous', 'Murder', 'Murder City', 'No referral', 'None', 'None of the above', 'None of these topics seem to be clearly defined as election campaigns or political subjects

## Output only the first class

In [24]:
df2 = pd.read_csv("../data/vicuna_4bit/lora/multilabel_without_context_v01/test_generic_test_0.csv")
vicuna_lora_multilabel_without_context_v01_df = df2#pd.concat([df1, df2])
# Cleaning and extracting multilabel classes
vicuna_lora_multilabel_without_context_v01_df['annotations'] = vicuna_lora_multilabel_without_context_v01_df['annotations'].apply(lambda x: extract_multilabel_list(x, classes))
vicuna_lora_multilabel_without_context_v01_df['testmultilabel_without_context_v01'] = vicuna_lora_multilabel_without_context_v01_df['testmultilabel_without_context_v01'].apply(lambda x: extract_multilabel_list_only_first_class(x, classes))

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

# Transform the labels into binary format
mlb = MultiLabelBinarizer(classes=classes)

# Prepare ground truth and predictions
y_true = mlb.fit_transform(vicuna_lora_multilabel_without_context_v01_df['annotations'])
y_pred = mlb.transform(vicuna_lora_multilabel_without_context_v01_df['testmultilabel_without_context_v01'])

#import pandas as pd
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(y_true, y_pred, output_dict=True, target_names=classes)

# Convert the report to a pandas DataFrame
df_report = pd.DataFrame(report).transpose()

print(df_report)

                                    precision    recall  f1-score  support
War/Terror                           0.953947  0.568627  0.712531    255.0
Conspiracy Theory                    0.478873  0.755556  0.586207     45.0
Education                            0.421053  0.615385  0.500000     13.0
Election Campaign                    0.750000  0.545455  0.631579     33.0
Environment                          0.700000  0.500000  0.583333     14.0
Government/Public                    0.781377  0.663230  0.717472    291.0
Health                               0.740741  0.434783  0.547945     46.0
Immigration/Integration              0.750000  0.250000  0.375000     36.0
Justice/Crime                        0.929293  0.671533  0.779661    137.0
Labor/Employment                     0.687500  0.392857  0.500000     28.0
Macroeconomics/Economic Regulation   0.833333  0.322581  0.465116     62.0
Media/Journalism                     0.750000  0.250000  0.375000     48.0
Religion                 

/media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/Documents/ip6-twitter-disinformation/.conda/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['Israel/Palestine', 'Propaganda'] will be ignored
  warnings.warn(
/media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/Documents/ip6-twitter-disinformation/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
df2 = pd.read_csv("../data/vicuna_4bit/lora/multilabel_with_rules_v02/test_generic_test_0.csv")
vicuna_lora_multilabel_without_context_v01_df = df2#pd.concat([df1, df2])
# Cleaning and extracting multilabel classes
vicuna_lora_multilabel_without_context_v01_df['annotations'] = vicuna_lora_multilabel_without_context_v01_df['annotations'].apply(lambda x: extract_multilabel_list(x, classes))
vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02'] = vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02'].apply(lambda x: extract_multilabel_list_only_first_class(x, classes))

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

# Transform the labels into binary format
mlb = MultiLabelBinarizer(classes=classes)

# Prepare ground truth and predictions
y_true = mlb.fit_transform(vicuna_lora_multilabel_without_context_v01_df['annotations'])
y_pred = mlb.transform(vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02'])

#import pandas as pd
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(y_true, y_pred, output_dict=True, target_names=classes)

# Convert the report to a pandas DataFrame
df_report = pd.DataFrame(report).transpose()

print(df_report)

                                    precision    recall  f1-score  support
War/Terror                           0.967949  0.592157  0.734793    255.0
Conspiracy Theory                    0.442857  0.688889  0.539130     45.0
Education                            0.363636  0.615385  0.457143     13.0
Election Campaign                    0.761905  0.484848  0.592593     33.0
Environment                          0.642857  0.642857  0.642857     14.0
Government/Public                    0.787755  0.663230  0.720149    291.0
Health                               0.818182  0.391304  0.529412     46.0
Immigration/Integration              0.750000  0.166667  0.272727     36.0
Justice/Crime                        0.911765  0.678832  0.778243    137.0
Labor/Employment                     0.647059  0.392857  0.488889     28.0
Macroeconomics/Economic Regulation   0.826087  0.306452  0.447059     62.0
Media/Journalism                     0.857143  0.250000  0.387097     48.0
Religion                 

In [20]:
df2

,id,campaign_name,text,annotations,prompt,context,normalized_tweet,testmultilabel_with_rules_v02
0,1144169368227635200,REA_0621,The Automobile Association said it is expectin...,[Macroeconomics/Economic Regulation],"Below is an instruction that describes a task,...",NaN,The Automobile Association said it is expectin...,"['Macroeconomics/Economic Regulation', 'Media/..."
1,1187637103850610688,REA_0621,"A severe flooding, triggered by heavy rains an...",[Environment],"Below is an instruction that describes a task,...",NaN,"A severe flooding , triggered by heavy rains a...","['Environment', 'Health', 'Immigration/Integra..."
2,1062269743749566464,GRU_202012,Militants of the Hayat Tahrir ash-Sham group p...,[War/Terror],"Below is an instruction that describes a task,...",NaN,Militants of the Hayat Tahrir ash-Sham group p...,"['War/Terror', 'Media/Journalism', 'War/Terror..."
3,1100425181946880000,REA_0621,PIC Amendment Bill for transparency adopted by...,[Government/Public],"Below is an instruction that describes a task,...",NaN,PIC Amendment Bill for transparency adopted by...,"['Others', 'Government/Public', 'Justice/Crime..."
4,1129575563029561344,UGANDA_0621,RT @CQvMyyB0YfvwrUrsaZ6KI7yqaJfSUDTrAI0joQhgMA...,[Government/Public],"Below is an instruction that describes a task,...",NaN,RT @USER =: ARMY CHARITY WORK :round_pushpin: ...,"['Others', 'War/Terror', 'Government/Public', ..."
...,...,...,...,...,...,...,...,...
995,1166999406190780417,REA_0621,Rwanda and Zimbabwe entering deal: A Memorandu...,[Macroeconomics/Economic Regulation],"Below is an instruction that describes a task,...",NaN,Rwanda and Zimbabwe entering deal : A Memorand...,"['Government/Public', 'Macroeconomics/Economic..."
996,887818127295361024,VENEZUELA_201901_2,Trump Supporters React to Unhinged Rosie THREA...,[Government/Public],"Below is an instruction that describes a task,...",NaN,Trump Supporters React to Unhinged Rosie THREA...,"['Government/Public', 'Justice/Crime', 'Media/..."
997,495089817923756032,IRA_202012,The American Aggression Enablement Act and the...,[Others],"Below is an instruction that describes a task,...",NaN,The American Aggression Enablement Act and the...,"['Conspiracy Theory', 'War/Terror'], #US #USA ..."
998,1154775632871477248,REA_0621,Body of a missing 4-year-old girl has been fou...,[Justice/Crime],"Below is an instruction that describes a task,...",NaN,Body of a missing 4 - year-old girl has been f...,"['Justice/Crime', 'War/Terror'], ‘Child Abuse ..."


In [15]:
df2

,id,campaign_name,text,annotations,prompt,context,normalized_tweet,testmultilabel_without_context_v01
0,1144169368227635200,REA_0621,The Automobile Association said it is expectin...,[Macroeconomics/Economic Regulation],"Below is an instruction that describes a task,...",NaN,The Automobile Association said it is expectin...,"['Macroeconomics/Economic Regulation', 'Transp..."
1,1187637103850610688,REA_0621,"A severe flooding, triggered by heavy rains an...",[Environment],"Below is an instruction that describes a task,...",NaN,"A severe flooding , triggered by heavy rains a...","['Environment', 'Health', 'Weather/Climate']"
2,1062269743749566464,GRU_202012,Militants of the Hayat Tahrir ash-Sham group p...,[War/Terror],"Below is an instruction that describes a task,...",NaN,Militants of the Hayat Tahrir ash-Sham group p...,"['War/Terror', 'Government/Public', 'Media/Jou..."
3,1100425181946880000,REA_0621,PIC Amendment Bill for transparency adopted by...,[Government/Public],"Below is an instruction that describes a task,...",NaN,PIC Amendment Bill for transparency adopted by...,"['Others', 'Government/Public', 'Media/Journal..."
4,1129575563029561344,UGANDA_0621,RT @CQvMyyB0YfvwrUrsaZ6KI7yqaJfSUDTrAI0joQhgMA...,[Government/Public],"Below is an instruction that describes a task,...",NaN,RT @USER =: ARMY CHARITY WORK :round_pushpin: ...,"['Government/Public', 'War/Terror', 'Health', ..."
...,...,...,...,...,...,...,...,...
995,1166999406190780417,REA_0621,Rwanda and Zimbabwe entering deal: A Memorandu...,[Macroeconomics/Economic Regulation],"Below is an instruction that describes a task,...",NaN,Rwanda and Zimbabwe entering deal : A Memorand...,"['Government/Public', 'Macroeconomics/Economic..."
996,887818127295361024,VENEZUELA_201901_2,Trump Supporters React to Unhinged Rosie THREA...,[Government/Public],"Below is an instruction that describes a task,...",NaN,Trump Supporters React to Unhinged Rosie THREA...,"['Government/Public', 'Justice/Crime', 'Media/..."
997,495089817923756032,IRA_202012,The American Aggression Enablement Act and the...,[Others],"Below is an instruction that describes a task,...",NaN,The American Aggression Enablement Act and the...,"['Conspiracy Theory', 'Government/Public', 'Wa..."
998,1154775632871477248,REA_0621,Body of a missing 4-year-old girl has been fou...,[Justice/Crime],"Below is an instruction that describes a task,...",NaN,Body of a missing 4 - year-old girl has been f...,"['Justice/Crime', 'War/Terror', 'Health', 'Jus..."


In [9]:
df2

,id,campaign_name,text,annotations,prompt,context,normalized_tweet,testmultilabel_with_rules_v02
0,1144169368227635200,REA_0621,The Automobile Association said it is expectin...,[Macroeconomics/Economic Regulation],"Below is an instruction that describes a task,...",NaN,The Automobile Association said it is expectin...,"['Macroeconomics/Economic Regulation', 'Media/..."
1,1187637103850610688,REA_0621,"A severe flooding, triggered by heavy rains an...",[Environment],"Below is an instruction that describes a task,...",NaN,"A severe flooding , triggered by heavy rains a...","['Environment', 'Health', 'Immigration/Integra..."
2,1062269743749566464,GRU_202012,Militants of the Hayat Tahrir ash-Sham group p...,[War/Terror],"Below is an instruction that describes a task,...",NaN,Militants of the Hayat Tahrir ash-Sham group p...,"['War/Terror', 'Media/Journalism', 'War/Terror..."
3,1100425181946880000,REA_0621,PIC Amendment Bill for transparency adopted by...,[Government/Public],"Below is an instruction that describes a task,...",NaN,PIC Amendment Bill for transparency adopted by...,"['Others', 'Government/Public', 'Justice/Crime..."
4,1129575563029561344,UGANDA_0621,RT @CQvMyyB0YfvwrUrsaZ6KI7yqaJfSUDTrAI0joQhgMA...,[Government/Public],"Below is an instruction that describes a task,...",NaN,RT @USER =: ARMY CHARITY WORK :round_pushpin: ...,"['Others', 'War/Terror', 'Government/Public', ..."
...,...,...,...,...,...,...,...,...
995,1166999406190780417,REA_0621,Rwanda and Zimbabwe entering deal: A Memorandu...,[Macroeconomics/Economic Regulation],"Below is an instruction that describes a task,...",NaN,Rwanda and Zimbabwe entering deal : A Memorand...,"['Government/Public', 'Macroeconomics/Economic..."
996,887818127295361024,VENEZUELA_201901_2,Trump Supporters React to Unhinged Rosie THREA...,[Government/Public],"Below is an instruction that describes a task,...",NaN,Trump Supporters React to Unhinged Rosie THREA...,"['Government/Public', 'Justice/Crime', 'Media/..."
997,495089817923756032,IRA_202012,The American Aggression Enablement Act and the...,[Others],"Below is an instruction that describes a task,...",NaN,The American Aggression Enablement Act and the...,"['Conspiracy Theory', 'War/Terror'], #US #USA ..."
998,1154775632871477248,REA_0621,Body of a missing 4-year-old girl has been fou...,[Justice/Crime],"Below is an instruction that describes a task,...",NaN,Body of a missing 4 - year-old girl has been f...,"['Justice/Crime', 'War/Terror'], ‘Child Abuse ..."


In [5]:
!python -m pip install seaborn

  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)


In [13]:
# Select only 'f1-score' and 'support' for each label
df_report_selected = df_report.loc[classes, ['f1-score', 'support']]

# Convert the DataFrame to a LaTeX table
latex_table = df_report_selected.to_latex()

print(latex_table)

\begin{tabular}{lrr}
\toprule
{} &  f1-score &  support \\
\midrule
War/Terror                         &  0.429419 &    440.0 \\
Conspiracy Theory                  &  0.434783 &     92.0 \\
Education                          &  0.382979 &     25.0 \\
Election Campaign                  &  0.390244 &     59.0 \\
Environment                        &  0.482759 &     28.0 \\
Government/Public                  &  0.489480 &    543.0 \\
Health                             &  0.438776 &     91.0 \\
Immigration/Integration            &  0.493333 &     76.0 \\
Justice/Crime                      &  0.400844 &    244.0 \\
Labor/Employment                   &  0.323810 &     47.0 \\
Macroeconomics/Economic Regulation &  0.307692 &    120.0 \\
Media/Journalism                   &  0.170426 &     86.0 \\
Religion                           &  0.185185 &     33.0 \\
Science/Technology                 &  0.089109 &     21.0 \\
Others                             &  0.501082 &    476.0 \\
\bottomrule
\end{

In [2]:
from tabulate import tabulate

models = [
    {
        "name": "No context - Classification only",
        "data": without_context_classification_only,
    },
    {
        "name": "No context - Elaboration first",
        "data": without_context_elaboration_first,
    },
    {
        "name": "With rules - Classification only",
        "data": with_rules_classification_only,
    },
    {
        "name": "With rules - Elaboration first",
        "data": with_rules_elaboration_first,
    }

]

for model in models:
    llm_utils.print_confusion_matrix(model["data"])

Confusion matrix for War/Terror:
[[56  9]
 [17 48]]

Confusion matrix for Conspiracy Theory:
[[54 11]
 [26 39]]

Confusion matrix for Education:
[[43 22]
 [ 9 56]]

Confusion matrix for Election Campaign:
[[42 23]
 [17 48]]

Confusion matrix for Environment:
[[36 29]
 [12 53]]

Confusion matrix for Government/Public:
[[32 33]
 [23 42]]

Confusion matrix for Health:
[[45 20]
 [16 49]]

Confusion matrix for Immigration/Integration:
[[52 13]
 [16 49]]

Confusion matrix for Justice/Crime:
[[28 37]
 [ 8 57]]

Confusion matrix for Labor/Employment:
[[47 18]
 [18 47]]

Confusion matrix for Macroeconomics/Economic Regulation:
[[63  2]
 [48 17]]

Confusion matrix for Media/Journalism:
[[31 94]
 [ 1  4]]

Confusion matrix for Religion:
[[97 31]
 [ 0  2]]

Confusion matrix for Science/Technology:
[]

Confusion matrix for War/Terror:
[[38 25]
 [ 6 59]]

Confusion matrix for Conspiracy Theory:
[[45 20]
 [26 38]]

Confusion matrix for Education:
[[24 41]
 [ 2 63]]

Confusion matrix for Election Camp

In [3]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 32.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 kB 30.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 66.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 39.3 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19581 sha256=55c8fa74348198d4344fe896c217ac8ef74d5fe65a57d200f2c5438bc8c94f12
  Stored in directory: /home/bruno/.var/app/com.visualstudio.code/cache/pip/wheels/82/35/dc/f88ec71edf2a5596bd72a8fa1b697277e0fcd3cde83048

In [9]:
!pip install streamlit-components

ERROR: Could not find a version that satisfies the requirement streamlit-components (from versions: none)
ERROR: No matching distribution found for streamlit-components


In [6]:
import streamlit as st
import pandas as pd

# Title
st.title("Classification Report and Confusion Matrix Viewer")

# Model selection
model_names = [model["name"] for model in models]
selected_model = st.sidebar.selectbox("Select model", model_names)

# Class selection
class_options = classes
selected_class = st.sidebar.selectbox("Select class", class_options)

# Get the selected data
selected_model_data = None
for model in models:
    if model["name"] == selected_model:
        selected_model_data = model["data"]
        break

selected_confusion_matrix = selected_model_data["confusion_matrices"][selected_class]
selected_classification_report = selected_model_data["classification_reports"][selected_class]

# Display the confusion matrix and classification report
st.header("Confusion Matrix")
st.write(selected_confusion_matrix)

st.header("Classification Report")
st.write(pd.DataFrame(selected_classification_report).transpose())

In [7]:
!streamlit run report_viewer.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: report_viewer.py


In [5]:
without_contex_only_classification_df[without_contex_only_classification_df["Macroeconomics/Economic Regulation_pred"].notna()].copy().annotations.to_list()[0:10]

["['War/Terror']",
 "['Others']",
 "['Macroeconomics/Economic Regulation']",
 "['Government/Public', 'Macroeconomics/Economic Regulation']",
 "['Government/Public', 'Macroeconomics/Economic Regulation', 'Media/Journalism', 'War/Terror']",
 "['Government/Public', 'Macroeconomics/Economic Regulation']",
 "['Others']",
 "['Government/Public', 'Macroeconomics/Economic Regulation']",
 "['Education', 'Media/Journalism']",
 "['Justice/Crime']"]

In [4]:
without_contex_only_classification_df[without_contex_only_classification_df["Macroeconomics/Economic Regulation_pred"].notna()].copy().text.to_list()[0:10]

['RT @Ben_Jonson_1985: #Feylak Ar-Rahman militants located in #EasternGhouta do not release #civilians from the fighting zone. The use of a l…',
 'Women’s March Organizer Wants to “Take Vagina Away” from Female Genital Mutilation Survivor https://t.co/MmBm9WD6Ux https://t.co/T9rBUrOySf',
 'IMF has demanded to take credit apartments of Ukrainians\nhttp://t.co/2hcqYxfQHc #Ukraine',
 'Libertarianism is nothing but liberty for exploiters, racists, oppressors and parasites. Socialism is liberty for the people; the working class. https://t.co/uyuugJzTpq',
 '🇮🇱🇵🇸Israel is now experiencing a moment of heightened political, economic, and military power is no excuse to avoid dealing with the Palestinian issue, writes #valdaiclub expert Amos Yadlin.\n\nhttps://t.co/7IPavmsHJh',
 'Following the power cut crisis faced by Zimbabwe where electricity is only available between 10pm and 6am, people have adjusted their daily routines to make the most out of the electricity when it is available.\n\nRead f